# Numerical exercises 06

From shell, inside `/code` directory:
- To compile: `make`

From shell, inside _this_ directory:
- To run the program: `./ex1.sh`

The scripts provided should take care of everything. The scripts set also the values of the input file. To change the input values the best option could be to modify the scripts themselves.

In [1]:
import numpy as np
from numpy import tanh, cosh, sinh, exp, sqrt
import matplotlib.pyplot as pl
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes    # zoomed up image libraries
from mpl_toolkits.axes_grid1.inset_locator import mark_inset

## Exercise 06.1: Ising Model simulation with Metropolis and Gibbs sampling

### Sampling the canonical ensemble probability distribution

The simulations of the molecular dynamics done in exercise 04 were performed in the microcanonical ensemble. In it, equilibrium states are identified by a collection of fixed extensive variables: the number of degrees of freedom $N$, the volume $V$, the internal energy $E$. Experimental constraints often make preferable a different choice. In particular, in the **canonical ensemble** the number of degrees of freedom $N$, the volume $V$, and the **temperature** $T$ are fixed. The thermodynamics of this system can be derived by the knowledge of the partition function or of the (Helmoltz) free energy associated to the ensemble $A = E -TS$:
$$
Q_N(V,T)=e^{-A(N,V;T)/k_B T} \quad \quad A(N,V,T) = -k_B T \ln Q_N(V,T)
$$
Calculating any thermodynamic property $\langle f \rangle$ with MC methods seems to be an impossible task without this knowledge:
$$
\langle f \rangle = \int \frac{d^{3N}q d^{3N} p}{h^{3N} N!} f(\vec q, \vec p ) \frac{ e^{-H(\vec q, \vec p)/k_B T}}{Q_N(V,T)},
$$
But wiht Metropolis algorithm we can in fact sample the probability distribution:
$$
\rho(\vec q, \vec p) = \frac{e^{-H(\vec q, \vec p)/k_B T}}{Q_N(V,T)}
$$
without knowing the normalization constant, that goes away in the acceptance step. 

### The Ising Model

In the simulation, we deal with a one-dimensional Ising model made up by $N=50$ spin degrees of freedom ($\pm 1$ are the only possible values) $\{s_i\}_{i=1}^N$, with periodic boundary condition: $s_{N+1}=s_1$. The Ising model is exactly resoluble, and its hamiltonian is given by, with $\mu_B=1$ and $k_B=1$:
$$
H = 
-J\sum_{i=1}^N s_i s_{i+1}
-\frac{h}{2}\sum_{i=1}^N (s_i + s_{i+1}),
$$
where $J$ is the coupling between nearest neighbors: at low enough temperatures, for positive values of $J$ the spins tend to orient themselves in the same direction (ferromagnetic behavior), for negative values they tend to orient themselves in opposite directions (anti-ferromagnetic behavior).

The partition function is given by a sum over all of the possible spin configurations:
$$
Q_N=\sum_{\{s_i\}} e^{-\beta H} = \lambda_1^N + \lambda_2^N, 
$$
with:
$$
\lambda_{1,2} = 
e^{\beta J} 
\cosh (\beta h) \pm \left[ e^{2\beta J} \cosh^2 (\beta h) - 2\sinh (2\beta J) \right]^{1/2}
$$

This is all we need to compute the analytical value of the thermodynamics quantities of interest. We will compare the theoretical results with the Monte Carlo outputs making use of the following relations:

$$ U(N,T,h) = \langle H \rangle $$

$$ C(N,T,h) = \frac{\partial U(N,T)}{\partial T} = \beta^2 \left(\langle H^2 \rangle -\langle H \rangle^2 \right) $$

$$ M(N,T,h) = k_B T \frac{\partial \ln Z}{\partial h} =
\left\langle \sum_{i=1}^N s_i \right\rangle $$

$$ \chi(N,T,h) = \frac{\partial M}{\partial h} =
\beta \left[ \left\langle \left( \sum_{i=1}^N s_i \right)^2 \right\rangle - \left\langle \sum_{i=1}^N s_i \right\rangle^2 \right] $$


In [ ]:
T=(0.5,0.95,2.)
eteo=[-J*( (tanh(J/t)) + (1/(tanh(J/t)))*((tanh(J/t))**Ns) )/( 1 + ((tanh(J/t))**Ns) )*Ns for t in T]
title=('First 200 points','last 500 points'); labteo=['expected <U>','','']

def graph_burn(ifMetro):
    if ifMetro: name="metro"
    else: name="gibbs"

    nburn=np.genfromtxt('results/burnIn.'+name+'_0.out',max_rows=1)
    burn0=np.genfromtxt('results/burnIn.'+name+'_0.out',skip_header=1,names='x,u')
    burn10=np.genfromtxt('results/burnIn.'+name+'_6.out',skip_header=1,names='x,u')
    burn20=np.genfromtxt('results/burnIn.'+name+'_20.out',skip_header=1,names='x,u')
    burn=(burn0,burn10,burn20); labT=['T='+str(temp) for temp in T]

    pl.figure(figsize=(15,4))
    for i in range(2):
        pl.subplot(1,2,i+1)
        for j in range(3): pl.plot(burn[j]['x'],burn[j]['u'],label=labT[j])
        for j in range(3): pl.axhline(y=eteo[j],color='k',linewidth=0.8,label=labteo[j])
        if i==0: pl.xlim(-1,200)
        elif i==1: pl.xlim(nburn-500,nburn)
        pl.legend(loc='best')
        pl.xlabel('$N_{burn}$')
        if i==0: pl.ylabel('$U$',labelpad=15)
        pl.title(title[i])
        pl.grid(True)
    pl.suptitle('Burn in, '+name+' sampling')
    pl.show()